# Assignment 1 — End-to-End Data Science with ChatGPT
**Dataset**: (name + link)
**Task**: (classification/regression)
**Goal Metric**: (F1 / ROC-AUC / RMSE / MAE)


In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
RANDOM_STATE=42
ARTIFACTS=Path('../artifacts'); ARTIFACTS.mkdir(exist_ok=True, parents=True)
RESULTS=Path('../results'); RESULTS.mkdir(exist_ok=True, parents=True)
print('Setup complete')


## 1) Load Data & Inspect

In [ ]:
df = pd.read_csv('/content/<your_file>.csv')  # TODO
df.head()


## 2) Split Train/Dev/Test

In [ ]:
from sklearn.model_selection import train_test_split
TARGET='<target>'
X=df.drop(columns=[TARGET]); y=df[TARGET]
X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.2,random_state=RANDOM_STATE)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp,y_temp,test_size=0.5,random_state=RANDOM_STATE)
X_train.shape, X_dev.shape, X_test.shape


## 3) Preprocess + Baseline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score, mean_squared_error

num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object','category','bool']).columns
preprocess = ColumnTransformer([
    ('num', Pipeline([('imp', SimpleImputer(strategy='median')), ('sc', StandardScaler(with_mean=False))]), num_cols),
    ('cat', Pipeline([('imp', SimpleImputer(strategy='most_frequent')), ('oh', OneHotEncoder(handle_unknown='ignore'))]), cat_cols)
])
IS_CLASS = y.nunique()<20 and y.dtype!='float'
model = LogisticRegression(max_iter=1000) if IS_CLASS else LinearRegression()
pipe = Pipeline([('prep', preprocess), ('model', model)])
pipe.fit(X_train, y_train)
pred = pipe.predict(X_dev)
score = f1_score(y_dev, pred, average='macro') if IS_CLASS else mean_squared_error(y_dev, pred, squared=False)
print('Baseline score:', score)


## 4) Save Artifacts & Summary

In [ ]:
import pandas as pd
pd.DataFrame({'metric':[float(score)]}).to_csv(RESULTS/'summary.csv', index=False)
print('Saved to results/summary.csv')
